In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

c:\Users\brand\Documents\NUS\Y4S2\HTX_xData\technical_test\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from flask import Flask

In [7]:
import pandas as pd

eg = pd.read_csv("../../cv-valid-dev.csv")
eg

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN
...,...,...,...,...,...,...,...,...
4071,cv-valid-dev/sample-004071.mp3,but they could never have taught him arabic,2,1,NaN,NaN,NaN,NaN
4072,cv-valid-dev/sample-004072.mp3,he decided to concentrate on more practical ma...,1,0,NaN,NaN,NaN,NaN
4073,cv-valid-dev/sample-004073.mp3,that's what i'm not supposed to say,2,0,thirties,male,us,NaN
4074,cv-valid-dev/sample-004074.mp3,just handling them made him feel better,3,0,NaN,NaN,NaN,NaN


In [4]:
sample_mp3_file_path = "../../sample-000000.mp3"

In [24]:
import librosa

ModuleNotFoundError: No module named 'librosa'

In [16]:
# # Load the MP3 file ensuring a 16kHz sample rate
# audio, sample_rate = librosa.load("temp_audio.mp3", sr=16000)

# # Calculate duration in seconds
# duration = librosa.get_duration(y=audio, sr=sample_rate)

librosa_sample_mp3_input, librosa_sample_mp3_sample_rate = librosa.load(sample_mp3_file_path, sr=16000)

NameError: name 'librosa' is not defined

In [5]:
import soundfile as sf

sample_mp3 = sf.read(sample_mp3_file_path)
sample_mp3_input, sample_mp3_sample_rate = sf.read(sample_mp3_file_path)

print(sample_mp3_input)
print(sample_mp3_sample_rate)

[ 0.00000000e+00  6.02674067e-17  6.65033890e-17 ... -2.24995560e-07
 -1.15900491e-07  1.59164927e-07]
48000


In [7]:
import torch

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# pad input values and return pt tensor
input_values = processor(sample_mp3_input, sampling_rate=16000, return_tensors="pt", padding='longest').input_values
input_values

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[-0.0002, -0.0002, -0.0002,  ..., -0.0002, -0.0002, -0.0002]])

In [8]:
input_values.shape

torch.Size([1, 243072])

In [9]:
# retrieve logits & take argmax
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)

In [11]:
logits, predicted_ids

(tensor([[[ 11.3729, -26.9658, -26.6519,  ...,  -4.6999,  -5.3485,  -7.4797],
          [ 11.3558, -26.9329, -26.6176,  ...,  -4.6864,  -5.2951,  -7.4361],
          [ 11.3806, -27.0193, -26.7050,  ...,  -4.6485,  -5.2828,  -7.4094],
          ...,
          [ 13.3263, -29.0487, -28.6795,  ...,  -5.9707,  -6.9709,  -8.1579],
          [ 12.9013, -28.7559, -28.4082,  ...,  -5.4768,  -6.4347,  -7.8917],
          [ 12.9958, -28.9273, -28.5808,  ...,  -5.7814,  -6.7785,  -7.9299]]],
        grad_fn=<ViewBackward0>),
 tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  

In [12]:
# transcribe
transcription = processor.decode(predicted_ids[0])

In [13]:
transcription

'HO O AORA PHA HAHO O'

In [14]:
type(transcription)

str

In [ ]:
# !pip install transformers
# !pip install datasets
import soundfile as sf
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# load pretrained model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")


librispeech_samples_ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# load audio
audio_input, sample_rate = sf.read(librispeech_samples_ds[0]["file"])

# pad input values and return pt tensor
input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

# INFERENCE

# retrieve logits & take argmax
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)

# transcribe
transcription = processor.decode(predicted_ids[0])

# FINE-TUNE

target_transcription = "A MAN SAID TO THE UNIVERSE I EXIST"

# encode labels
with processor.as_target_processor():
  labels = processor(target_transcription, return_tensors="pt").input_ids

# compute loss by passing labels
loss = model(input_values, labels=labels).loss
loss.backward()

In [19]:
import requests, os

In [27]:
x = load_dataset("../data/common_voice/cv-other-dev")
x

DatasetDict({
    validation: Dataset({
        features: ['audio'],
        num_rows: 3022
    })
})

In [28]:
x.keys()

dict_keys(['validation'])

In [32]:
x['validation']

Dataset({
    features: ['audio'],
    num_rows: 3022
})

In [33]:
type(x['validation'])

datasets.arrow_dataset.Dataset

In [26]:
type(load_dataset("../data/common_voice/cv-other-dev"))

datasets.dataset_dict.DatasetDict

In [ ]:
# df = pd.read_csv("../data/common_voice/cv-valid-dev/cv-valid-dev.csv")
import pandas as pd
df = pd.read_csv("../data/common_voice/cv-other-dev.csv")
df

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-other-dev/sample-000000.mp3,she composed an emotional song in her bedroom,0,0,NaN,NaN,NaN,NaN
1,cv-other-dev/sample-000001.mp3,i could die happily and that made me feel good,0,0,NaN,NaN,NaN,NaN
2,cv-other-dev/sample-000002.mp3,what are you doing up here then,0,0,NaN,NaN,NaN,NaN
3,cv-other-dev/sample-000003.mp3,hundreds of observers saw the flame that night...,0,0,NaN,NaN,NaN,NaN
4,cv-other-dev/sample-000004.mp3,are you going to live with your mother,0,0,thirties,male,us,NaN
...,...,...,...,...,...,...,...,...
3017,cv-other-dev/sample-003017.mp3,you forgot to pack it,0,0,thirties,male,scotland,NaN
3018,cv-other-dev/sample-003018.mp3,a tow truck came to clear up the wreckage,0,0,twenties,male,england,NaN
3019,cv-other-dev/sample-003019.mp3,i am upset that they get to have all the time ...,0,0,twenties,female,NaN,NaN
3020,cv-other-dev/sample-003020.mp3,i made a few changes don't worry,0,0,NaN,NaN,NaN,NaN


In [22]:
df.loc[0]['filename']

'cv-other-dev/sample-000000.mp3'

In [23]:
file_path = os.path.join("../data/common_voice/cv-other-dev", df.loc[0]['filename'])
file_path

'../data/common_voice/cv-other-dev\\cv-other-dev/sample-000000.mp3'

In [25]:
sf.read(file_path)

(array([ 0.00000000e+00,  1.13895465e-13, -8.01868879e-14, ...,
         2.00181580e-06,  8.00205498e-07, -9.74069053e-07], shape=(154368,)),
 48000)